# Evolve individual binaries 🐞

It can be extremely useful to evolve a single binary from specific initial conditions or to start at a specific evolutionary state.
This is useful for debugging binaries and checking if your custom steps or flow are correctly working.

This tutorial will cover:

- How to initialise a zero-age main sequence binary (ZAMS).
- How to re-evolve a binary in an existing population.
- How to evolve a binary from specific initial states.


## Evolve a ZAMS binary

To evolve a binary from ZAMS, we will:


1. Load the standard simulation properties.
2. Load the steps. 
3. Initialise the binary.
4. Evolve it.

In [1]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

'./population_params.ini'

In [10]:
# load the function to load the simulation properties from the ini file
from posydon.popsyn.io import simprop_kwargs_from_ini
from posydon.binary_evol.simulationproperties import SimulationProperties

# Load the simulation properties from the default ini file. 
sim_kwargs = simprop_kwargs_from_ini('population_params.ini')
# manually add the metallicity to each step that requires it
metallicity = {'metallicity':1}

sim_kwargs['step_HMS_HMS'][1].update(metallicity)
sim_kwargs['step_CO_HeMS'][1].update(metallicity)
sim_kwargs['step_CO_HMS_RLO'][1].update(metallicity)
sim_kwargs['step_CO_HeMS_RLO'][1].update(metallicity)
sim_kwargs['step_detached'][1].update(metallicity)
sim_kwargs['step_disrupted'][1].update(metallicity)
sim_kwargs['step_merged'][1].update(metallicity)
sim_kwargs['step_initially_single'][1].update(metallicity)

sim_prop = SimulationProperties(**sim_kwargs)
# Load the steps and required data
sim_prop.load_steps(verbose=True)

STEP NAME           STEP FUNCTION            KWARGS
flow (<function flow_chart at 0x11f7bfec0>, {})
step_HMS_HMS (<class 'posydon.binary_evol.MESA.step_mesa.MS_MS_step'>, {'interpolation_path': None, 'interpolation_filename': None, 'interpolation_method': 'linear3c_kNN', 'save_initial_conditions': True, 'track_interpolation': False, 'stop_method': 'stop_at_max_time', 'stop_star': 'star_1', 'stop_var_name': None, 'stop_value': None, 'stop_interpolate': True, 'verbose': False, 'metallicity': 1})
step_CO_HeMS (<class 'posydon.binary_evol.MESA.step_mesa.CO_HeMS_step'>, {'interpolation_path': None, 'interpolation_filename': None, 'interpolation_method': 'linear3c_kNN', 'save_initial_conditions': True, 'track_interpolation': False, 'stop_method': 'stop_at_max_time', 'stop_star': 'star_1', 'stop_var_name': None, 'stop_value': None, 'stop_interpolate': True, 'verbose': False, 'metallicity': 1})
step_CO_HMS_RLO (<class 'posydon.binary_evol.MESA.step_mesa.CO_HMS_RLO_step'>, {'interpolation_path'

In [12]:
# load the binary and single star classes
from posydon.binary_evol.singlestar import SingleStar
from posydon.binary_evol.binarystar import BinaryStar

In [46]:
STAR1 = SingleStar(**{'mass': 30.782576, 
                      'state': 'H-rich_Core_H_burning'})
STAR2 = SingleStar(**{'mass':20.273864,
                      'state': 'H-rich_Core_H_burning'})

BINARY = BinaryStar(STAR1, STAR2,  
                    **{'time': 0.0, 'state': 'detached', 'event': 'ZAMS', 'orbital_period':3513.150157, 'eccentricity': 0.0},
                    properties = sim_prop)

In [54]:
# Note: depending on the evolution of the binary, you might get some warnings about the Roche lobe calculation.
BINARY.evolve()

You're now able to access the binary and evolutionary information. Using the function, `BinaryStar.to_df()` we can inspect the evolution of the systm.


In [57]:
# we will add the step names to the dataframe
col = ['time', 'step_names', 'state', 'event', 'orbital_period', 'eccentricity', 'S1_state', 'S2_state', 'S1_mass', 'S2_mass']

BINARY.to_df(extra_columns={'step_names':'string'})[col]

,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
NaN,0.000000e+00,initial_cond,detached,ZAMS,3513.150157,0.000000,H-rich_Core_H_burning,H-rich_Core_H_burning,30.782576,20.273864
NaN,6.735757e+06,step_HMS_HMS,detached,CC1,7695.358380,0.000000,stripped_He_Central_C_depletion,H-rich_Core_H_burning,13.675696,19.854313
NaN,6.735757e+06,step_SN,detached,<NA>,44234.704065,0.688255,BH,H-rich_Core_H_burning,13.071946,19.854313
NaN,1.016206e+07,step_detached,detached,CC2,86127.315626,0.675650,BH,H-rich_Central_C_depletion,13.071946,9.851849
NaN,1.016206e+07,step_SN,disrupted,<NA>,NaN,NaN,BH,BH,13.071946,8.807030
NaN,1.016206e+07,step_end,disrupted,END,NaN,NaN,BH,BH,13.071946,8.807030


### Re-evolving the Binary

The above binary might have been disrupted in the first or second SN due to the strength of the supernova kick.
You can restore the binary completely or to a specific state and re-evolve it.

In [62]:
# restore to the original state
BINARY.restore()
BINARY.to_df()

,state,event,time,separation,orbital_period,eccentricity,rl_relative_overflow_1,rl_relative_overflow_2,lg_mtransfer_rate,mass_transfer_case,trap_radius,acc_radius,t_sync_rad_1,t_sync_conv_1,t_sync_rad_2,t_sync_conv_2,nearest_neighbour_distance,V_sys_x,V_sys_y,V_sys_z,S1_state,S1_metallicity,S1_mass,S1_log_R,S1_log_L,S1_lg_mdot,S1_lg_system_mdot,S1_lg_wind_mdot,S1_he_core_mass,S1_he_core_radius,S1_c_core_mass,S1_c_core_radius,S1_o_core_mass,S1_o_core_radius,S1_co_core_mass,S1_co_core_radius,S1_center_h1,S1_center_he4,S1_center_c12,S1_center_n14,...,S2_o_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_center_c12,S2_center_n14,S2_center_o16,S2_surface_h1,S2_surface_he4,S2_surface_c12,S2_surface_n14,S2_surface_o16,S2_log_LH,S2_log_LHe,S2_log_LZ,S2_log_Lnuc,S2_c12_c12,S2_center_gamma,S2_avg_c_in_c_core,S2_surf_avg_omega,S2_surf_avg_omega_div_omega_crit,S2_total_moment_of_inertia,S2_log_total_angular_momentum,S2_spin,S2_conv_env_top_mass,S2_conv_env_bot_mass,S2_conv_env_top_radius,S2_conv_env_bot_radius,S2_conv_env_turnover_time_g,S2_conv_env_turnover_time_l_b,S2_conv_env_turnover_time_l_t,S2_envelope_binding_energy,S2_mass_conv_reg_fortides,S2_thickness_conv_reg_fortides,S2_radius_conv_reg_fortides,S2_lambda_CE_1cent,S2_lambda_CE_10cent,S2_lambda_CE_30cent,S2_lambda_CE_pure_He_star_10cent
binary_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,detached,ZAMS,0.0,NaN,3513.150157,0.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[None, None, None]",0.0,0.0,0.0,H-rich_Core_H_burning,NaN,30.782576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
BINARY.evolve()

,state,event,time,separation,orbital_period,eccentricity,rl_relative_overflow_1,rl_relative_overflow_2,lg_mtransfer_rate,mass_transfer_case,trap_radius,acc_radius,t_sync_rad_1,t_sync_conv_1,t_sync_rad_2,t_sync_conv_2,nearest_neighbour_distance,V_sys_x,V_sys_y,V_sys_z,S1_state,S1_metallicity,S1_mass,S1_log_R,S1_log_L,S1_lg_mdot,S1_lg_system_mdot,S1_lg_wind_mdot,S1_he_core_mass,S1_he_core_radius,S1_c_core_mass,S1_c_core_radius,S1_o_core_mass,S1_o_core_radius,S1_co_core_mass,S1_co_core_radius,S1_center_h1,S1_center_he4,S1_center_c12,S1_center_n14,...,S2_o_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_center_c12,S2_center_n14,S2_center_o16,S2_surface_h1,S2_surface_he4,S2_surface_c12,S2_surface_n14,S2_surface_o16,S2_log_LH,S2_log_LHe,S2_log_LZ,S2_log_Lnuc,S2_c12_c12,S2_center_gamma,S2_avg_c_in_c_core,S2_surf_avg_omega,S2_surf_avg_omega_div_omega_crit,S2_total_moment_of_inertia,S2_log_total_angular_momentum,S2_spin,S2_conv_env_top_mass,S2_conv_env_bot_mass,S2_conv_env_top_radius,S2_conv_env_bot_radius,S2_conv_env_turnover_time_g,S2_conv_env_turnover_time_l_b,S2_conv_env_turnover_time_l_t,S2_envelope_binding_energy,S2_mass_conv_reg_fortides,S2_thickness_conv_reg_fortides,S2_radius_conv_reg_fortides,S2_lambda_CE_1cent,S2_lambda_CE_10cent,S2_lambda_CE_30cent,S2_lambda_CE_pure_He_star_10cent
binary_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,detached,ZAMS,0.0,NaN,3513.150157,0.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[None, None, None]",0.0,0.0,0.0,H-rich_Core_H_burning,NaN,30.782576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Even thought we reset the binary, the natal kick properties stored in the natal kick array are not reset. This is because we want to be able to re-evolve the binary to the same final state. In case you want to reset the natal kick array values, then set the list to `None` values.

In [63]:
# kick magnitude km/s, azimuthal angle rad, polar angle rad and mean anomaly rad (TODO check documentation)
print(BINARY.star_1.natal_kick_array)
print(BINARY.star_2.natal_kick_array)

[32.58749914542419, 3.0465057278083143, 1.689468461187366, 0.6240980411520373]
[49.8291439058427, 4.154565117887979, 2.2011009713336844, 0.06586326638901714]


In [ ]:
# This will be exactly the same evolution as the previous one
BINARY.to_df(extra_columns={'step_names':'string'})[col]

#### Changing the kick

We will restore the `BINARY` to its initial state and increase the first kick velocity to disrupt the binary after the first supernova.

In [64]:
BINARY.restore()
BINARY.to_df()

,state,event,time,separation,orbital_period,eccentricity,rl_relative_overflow_1,rl_relative_overflow_2,lg_mtransfer_rate,mass_transfer_case,trap_radius,acc_radius,t_sync_rad_1,t_sync_conv_1,t_sync_rad_2,t_sync_conv_2,nearest_neighbour_distance,V_sys_x,V_sys_y,V_sys_z,S1_state,S1_metallicity,S1_mass,S1_log_R,S1_log_L,S1_lg_mdot,S1_lg_system_mdot,S1_lg_wind_mdot,S1_he_core_mass,S1_he_core_radius,S1_c_core_mass,S1_c_core_radius,S1_o_core_mass,S1_o_core_radius,S1_co_core_mass,S1_co_core_radius,S1_center_h1,S1_center_he4,S1_center_c12,S1_center_n14,...,S2_o_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_center_c12,S2_center_n14,S2_center_o16,S2_surface_h1,S2_surface_he4,S2_surface_c12,S2_surface_n14,S2_surface_o16,S2_log_LH,S2_log_LHe,S2_log_LZ,S2_log_Lnuc,S2_c12_c12,S2_center_gamma,S2_avg_c_in_c_core,S2_surf_avg_omega,S2_surf_avg_omega_div_omega_crit,S2_total_moment_of_inertia,S2_log_total_angular_momentum,S2_spin,S2_conv_env_top_mass,S2_conv_env_bot_mass,S2_conv_env_top_radius,S2_conv_env_bot_radius,S2_conv_env_turnover_time_g,S2_conv_env_turnover_time_l_b,S2_conv_env_turnover_time_l_t,S2_envelope_binding_energy,S2_mass_conv_reg_fortides,S2_thickness_conv_reg_fortides,S2_radius_conv_reg_fortides,S2_lambda_CE_1cent,S2_lambda_CE_10cent,S2_lambda_CE_30cent,S2_lambda_CE_pure_He_star_10cent
binary_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,detached,ZAMS,0.0,NaN,3513.150157,0.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[None, None, None]",0.0,0.0,0.0,H-rich_Core_H_burning,NaN,30.782576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# [velocity, azimuthal angle, polar angle, phase]
BINARY.star_1.natal_kick_array = [1000., 0.858129274334538, 1.9157148786534735, 1.8675467897282945]
BINARY.star_2.natal_kick_array = [None, None, None, None] # default
print(BINARY.star_1.natal_kick_array)
print(BINARY.star_2.natal_kick_array)

[1000.0, 0.858129274334538, 1.9157148786534735, 1.8675467897282945]
[None, None, None, None]


In [67]:
BINARY.evolve()
BINARY.to_df(extra_columns={'step_names':'string'})[col]

/Users/max/Documents/POSYDON/posydon/binary_evol/DT/step_detached.py:1690: RuntimeWarning: invalid value encountered in log10
  current = np.log10(


,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
NaN,0.000000e+00,initial_cond,detached,ZAMS,3513.150157,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,30.782576,20.273864
NaN,6.735757e+06,step_HMS_HMS,detached,CC1,7695.358380,0.0,stripped_He_Central_C_depletion,H-rich_Core_H_burning,13.675696,19.854313
NaN,6.735757e+06,step_SN,disrupted,<NA>,NaN,NaN,BH,H-rich_Core_H_burning,13.071946,19.854313
NaN,1.016206e+07,step_disrupted,disrupted,CC2,NaN,NaN,BH,H-rich_Central_C_depletion,13.071946,9.851849
NaN,1.016206e+07,step_SN,disrupted,<NA>,NaN,NaN,BH,BH,13.071946,8.807030
NaN,1.016206e+07,step_end,disrupted,END,NaN,NaN,BH,BH,13.071946,8.807030


#### Evolve from specific step

You can provide the row number (starting at 0), to reset the binary to that specific evolutionary phase.
This can be useful for debugging a specific evolutionary state.

In [69]:
BINARY.restore(3)
BINARY.to_df()

## Loading a binary from an existing population

Assume we run the population somewhere else and you have the h5 file associated with it, and you want to load the population in memory and re-evolve a binary. 
We will have to load the `population_params.ini` file associated with the Population file to re-evolve the binary in the same way.

We will use the `BBH_contact.h5` file in the example dataset and use the standard `population_params.ini` SimulationProperties from earlier in this notebook.

In [72]:
from posydon.popsyn.synthetic_population import Population
from posydon.config import PATH_TO_POSYDON_DATA

data_path = f'{PATH_TO_POSYDON_DATA}/POSYDON_data/tutorial/population-synthesis/examples/'

pop = Population('BBH_contact.h5')

In [143]:
# Because the original population contains two extre columns. These have to be defined manually here.
# Otherwise, the reseting of the binaries fails.
BINARY = BinaryStar.from_df(pop.history[0],
                            extra_columns={'step_names':'string','step_times':'float'})

# you have to set the simulation properties for the binary
BINARY.properties = sim_prop

# Set the natal kick arrays from the oneline to get the exact same evolution as the original binary
BINARY.star_1.natal_kick_array = pop.oneline[0][['S1_natal_kick_array_0', 'S1_natal_kick_array_1', 'S1_natal_kick_array_2', 'S1_natal_kick_array_3']].values.flatten()
BINARY.star_2.natal_kick_array = pop.oneline[0][['S2_natal_kick_array_0', 'S2_natal_kick_array_1', 'S2_natal_kick_array_2', 'S2_natal_kick_array_3']].values.flatten()

In [149]:
BINARY.to_df(extra_columns={'step_names':'string'})[col]

,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
0,0.000000e+00,initial_cond,detached,ZAMS,1.955424,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,36.717651,30.939512
0,4.550356e+06,step_HMS_HMS,contact,oCE1,2.259595,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,25.314644,34.652791
0,4.550356e+06,step_CE,merged,oMerging1,2.259595,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,25.314644,34.652791
0,5.890307e+06,step_merged,merged,CC1,NaN,NaN,stripped_He_Central_C_depletion,massless_remnant,25.373960,0.000000
0,5.890307e+06,step_SN,merged,<NA>,NaN,NaN,BH,massless_remnant,24.873960,0.000000
0,5.890307e+06,step_end,merged,END,NaN,NaN,BH,massless_remnant,24.873960,0.000000


In [146]:
BINARY.restore()

In [148]:
BINARY.evolve()
BINARY.to_df(extra_columns={'step_names':'string'})[col]

,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
0,0.000000e+00,initial_cond,detached,ZAMS,1.955424,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,36.717651,30.939512
0,4.550356e+06,step_HMS_HMS,contact,oCE1,2.259595,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,25.314644,34.652791
0,4.550356e+06,step_CE,merged,oMerging1,2.259595,0.0,H-rich_Core_H_burning,H-rich_Core_H_burning,25.314644,34.652791
0,5.890307e+06,step_merged,merged,CC1,NaN,NaN,stripped_He_Central_C_depletion,massless_remnant,25.373960,0.000000
0,5.890307e+06,step_SN,merged,<NA>,NaN,NaN,BH,massless_remnant,24.873960,0.000000
0,5.890307e+06,step_end,merged,END,NaN,NaN,BH,massless_remnant,24.873960,0.000000


## Evolving a Binary Star starting from an arbitrary state

If you want to evolve a custom binary star, you can do so by crafting yourself the `BinaryStar` object and providing the `SimulationProperties` object. For example, let's evolve a neutron star with a low mass helium star in Roche lobe overflow.
And start a binary in the detached step, which requires a bit of additional input data.

We will use the standard SimulationProperties, loaded earlier in this notebook.


<div class="alert alert-warning"><b>Caution!</b> 

While you can evolve a binary from an arbitrary state, you will need to provide data on the internal structure of the star, if you're starting in the detached step.
Otherwise, the matching to a single star model will not work!
</div>

In [157]:
# we intiialise the binary to be tight and Roche lobe overflowing.
binary = BinaryStar(star_1 = SingleStar(**{'state' : 'NS',
                                           'mass' : 1.1,
                                           'spin' : 0.,}),
                    star_2 = SingleStar(**{'state' : 'H-rich_Core_H_burning',
                                           'mass' : 2.5,
                                           'natal_kick_array' : [10., 0., 0., 0.]}),
                    **{'time' : 0.,
                       'state' : 'RLO2',
                       'event' : 'oRLO2',
                       'orbital_period' : 1.,
                       'eccentricity' : 0.},
                    properties = sim_prop,
                    )
binary.evolve()
binary.to_df(extra_columns={'step_names':'string'})[col]

,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
NaN,0.000000e+00,initial_cond,RLO2,oRLO2,1.000000,0.0,NS,H-rich_Core_H_burning,1.100000,2.500000
NaN,2.352552e+08,step_CO_HMS_RLO,RLO2,oCE2,0.085162,0.0,NS,H-rich_Core_H_burning,1.893069,0.164589
NaN,2.352552e+08,step_CE,merged,oMerging2,0.085162,0.0,NS,H-rich_Core_H_burning,1.893069,0.164589
NaN,2.352552e+08,step_merged,merged,<NA>,NaN,NaN,NS,massless_remnant,1.893069,0.000000
NaN,2.352552e+08,step_end,merged,END,NaN,NaN,NS,massless_remnant,1.893069,0.000000


In [158]:
# we perform the same evolution, but with a stripped helium star to go to the CO_HeMS_RLO grid
binary = BinaryStar(star_1 = SingleStar(**{'state' : 'NS',
                                           'mass' : 1.1,
                                           'spin' : 0.,}),
                    star_2 = SingleStar(**{'state' : 'stripped_He_Core_He_burning',
                                           'mass' : 2.5,
                                           'natal_kick_array' : [10., 0., 0., 0.]}),
                    **{'time' : 0.,
                       'state' : 'RLO2',
                       'event' : 'oRLO2',
                       'orbital_period' : 1.,
                       'eccentricity' : 0.},
                    properties = sim_prop,
                    )
binary.evolve()
binary.to_df(extra_columns={'step_names':'string'})[col]

,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
NaN,0.000000e+00,initial_cond,RLO2,oRLO2,1.000000,0.000000,NS,stripped_He_Core_He_burning,1.100000,2.500000
NaN,2.271486e+04,step_CO_HeMS_RLO,RLO2,CC2,0.630731,0.000000,NS,stripped_He_Central_C_depletion,1.100794,1.748416
NaN,2.271486e+04,step_SN,detached,<NA>,1.124790,0.276011,NS,NS,1.100794,1.260782
NaN,1.380000e+10,step_dco,detached,maxtime,0.960102,0.241684,NS,NS,1.100794,1.260782
NaN,1.380000e+10,step_end,detached,END,0.960102,0.241684,NS,NS,1.100794,1.260782


We will change the binary to a much wider system and have it start in the detached state.
This requires additional starting information to allow the star to be matched to the single star grids.


In [163]:
from posydon.utils.constants import Zsun
from posydon.utils.common_functions import orbital_separation_from_period

import numpy as np
Z = 1.0

# Setup the central abundances
zams_table = {2.: 2.915e-01,
                      1.: 2.703e-01,
                      0.45: 2.586e-01,
                      0.2: 2.533e-01,
                      0.1: 2.511e-01,
                      0.01: 2.492e-01,
                      0.001: 2.49e-01,
                      0.0001: 2.49e-01}

Y = zams_table[Z]
Z = Z*Zsun
X = 1 - Y - Z

In [164]:
STAR1 = SingleStar(**{'mass':1.2,
                      'state': 'NS'})

STAR2 = SingleStar(**{'mass': 17.782576, 
                      'state': 'H-rich_Core_H_burning',
                      # add the metallicity and central abundances
                      'metallicity':Z, 
                      'center_h1':X,
                      'center_he4':Y, 
                      # add a numerical value for the radius
                      'log_R': np.nan,
                      # add the helium core mass
                      'he_core_mass': 0.0,
                      })

binary = BinaryStar(STAR1, STAR2,
                    **{'time' : 0.,
                       'state' : 'detached',
                       'event' : None,
                       'orbital_period' : 5000.,
                        # calculate the separation; current bug that the separation is not automatically calculated if orbital period is given
                       'separation': orbital_separation_from_period(5000., 17.782576, 1.2),
                       'eccentricity' : 0.},
                    properties = sim_prop,
                    )
binary.evolve()
binary.to_df(extra_columns={'step_names':'string'})[col]

,time,step_names,state,event,orbital_period,eccentricity,S1_state,S2_state,S1_mass,S2_mass
binary_index,,,,,,,,,,
NaN,0.000000e+00,initial_cond,detached,<NA>,5000.000000,0.0,NS,H-rich_Core_H_burning,1.20000,17.782576
NaN,1.116368e+07,step_detached,RLO2,oRLO2,2504.595474,0.0,NS,H-rich_Core_He_burning,1.20000,15.838751
NaN,1.116463e+07,step_CO_HMS_RLO,RLO2,oCE2,1983.546394,0.0,NS,H-rich_Core_He_burning,1.20001,15.532967
NaN,1.116463e+07,step_CE,merged,oMerging2,1983.546394,0.0,NS,H-rich_Core_He_burning,1.20001,15.532967
NaN,1.116463e+07,step_merged,merged,<NA>,NaN,NaN,NS,massless_remnant,1.20001,0.000000
NaN,1.116463e+07,step_end,merged,END,NaN,NaN,NS,massless_remnant,1.20001,0.000000
